In [415]:
#사용할 패키지들 임포트
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


#먼저 사용할 파일들을 먼저 가져오기.
sale19 = pd.read_csv('Data/input_Data/서울시_상권분석서비스(매출-행정동)_2019년.csv', encoding = 'cp949')
sale20 = pd.read_csv('Data/input_Data/서울시_상권분석서비스(매출-행정동)_2020년.csv', encoding = 'cp949')
sale21 = pd.read_csv('Data/input_Data/서울시_상권분석서비스(매출-행정동)_2021년.csv', encoding = 'cp949')
sale22 = pd.read_csv('Data/input_Data/서울시_상권분석서비스(매출-행정동)_2022년.csv', encoding = 'cp949')
sale234 = pd.read_csv('Data/input_Data/서울시-상권분석서비스(매출-행정동)_2023년_2024년.csv', encoding = 'cp949')
consump = pd.read_csv('Data/input_Data/서울시 상권분석서비스(소득소비-행정동).csv', encoding = 'cp949')
#한글패치
plt.rcParams.update({'figure.dpi':'150'})
plt.rcParams.update({'font.family':'Malgun Gothic'})
#팔레트 색상 지정
colors = sns.color_palette('rocket_r', 15)


In [416]:
# 데이터들을 concat을 사용해서 병합후 total_sales변수에 저장

total_sales = pd.concat([sale19, sale20, sale21, sale22, sale234], ignore_index=True)
total_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368339 entries, 0 to 368338
Data columns (total 53 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   기준_년분기_코드        368339 non-null  int64 
 1   행정동_코드           368339 non-null  int64 
 2   행정동_코드_명         368339 non-null  object
 3   서비스_업종_코드        368339 non-null  object
 4   서비스_업종_코드_명      368339 non-null  object
 5   당월_매출_금액         368339 non-null  int64 
 6   당월_매출_건수         368339 non-null  int64 
 7   주중_매출_금액         368339 non-null  int64 
 8   주말_매출_금액         368339 non-null  int64 
 9   월요일_매출_금액        368339 non-null  int64 
 10  화요일_매출_금액        368339 non-null  int64 
 11  수요일_매출_금액        368339 non-null  int64 
 12  목요일_매출_금액        368339 non-null  int64 
 13  금요일_매출_금액        368339 non-null  int64 
 14  토요일_매출_금액        368339 non-null  int64 
 15  일요일_매출_금액        368339 non-null  int64 
 16  시간대_00~06_매출_금액  368339 non-null  int64 
 17  시간대_06~11_

In [417]:
# 외식업 사업을 하게 된다면 내가 알아야 할 정보들을 찾을 것이기 때문에 외식업만 찾아온다. 
# "서비스_업종_코드"가 CS1---- 으로 시작하는 벨류들은 외식업이라는 의미이기때문에 CS1으로 시작하는 데이터들만 찾아 온 후 
dining_sales = total_sales[total_sales['서비스_업종_코드'].str.startswith('CS1')]
#서비스_업종_코드 부분이 cs1만 있는거 확인하기.
dining_sales.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
4,20191,11380570,대조동,CS100010,커피-음료,779058661,129239,507363643,271695018,85425895,...,31317,14172,54266,74973,3248,44964,33514,25114,17579,4820
20,20191,11380631,신사1동,CS100009,호프-간이주점,89854690,2644,61977454,27877236,8666914,...,346,1797,1361,1084,12,690,516,484,501,242
26,20191,11380580,응암1동,CS100004,양식음식점,178109358,8852,110526577,67582781,10572631,...,3651,581,3556,5243,101,2077,3012,2090,1040,478
30,20191,11380600,응암3동,CS100003,일식음식점,6905046,515,6190260,714786,1077435,...,251,0,169,346,11,292,127,74,11,0
32,20191,11380640,증산동,CS100008,분식전문점,3000000,227,2929395,70605,994105,...,96,0,73,154,0,36,49,35,63,44


In [418]:
# 행정동별로/업종별/년분기별로 봐야하기 때문에 정렬
dining_sales = dining_sales.sort_values(['행정동_코드', '서비스_업종_코드_명', '기준_년분기_코드']
                                        ,ascending=[True, True, True]).reset_index(drop = True)


# 쓰지 않을 컬럼들 drop 함수를 사용해서 제거
dining_sales = dining_sales.drop(columns=[
    
    "주중_매출_건수", "주말_매출_건수", "월요일_매출_건수", "화요일_매출_건수",
    "수요일_매출_건수", "목요일_매출_건수", "금요일_매출_건수", "토요일_매출_건수",
    "일요일_매출_건수", "시간대_건수~06_매출_건수", "시간대_건수~11_매출_건수",
    "시간대_건수~14_매출_건수", "시간대_건수~17_매출_건수", "시간대_건수~21_매출_건수",
    "시간대_건수~24_매출_건수", "남성_매출_건수", "여성_매출_건수", 
    "연령대_10_매출_건수", "연령대_20_매출_건수", "연령대_30_매출_건수", 
    "연령대_40_매출_건수", "연령대_50_매출_건수", "연령대_60_이상_매출_건수","당월_매출_건수"
], axis = 1)

dining_sales.columns

Index(['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드', '서비스_업종_코드_명',
       '당월_매출_금액', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액',
       '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액',
       '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액'],
      dtype='object')

In [419]:
#컬럼명 변경하기
dining_sales = dining_sales.rename(columns={
    "월요일_매출_금액": "월요일매출",
    "화요일_매출_금액": "화요일매출",
    "수요일_매출_금액": "수요일매출",
    "목요일_매출_금액": "목요일매출",
    "금요일_매출_금액": "금요일매출",
    "토요일_매출_금액": "토요일매출",
    "일요일_매출_금액": "일요일매출",
    "당월_매출_금액":"당월매출",
    "연령대_10_매출_금액": "10대_매출금액",
    "연령대_20_매출_금액": "20대_매출금액",
    "연령대_30_매출_금액": "30대_매출금액",
    "연령대_40_매출_금액": "40대_매출금액",
    "연령대_50_매출_금액": "50대_매출금액",
    "연령대_60_이상_매출_금액": "60대_매출금액",
    "남성_매출_금액": "남성매출",
    "여성_매출_금액": "여성매출",
    "행정동_코드_명": "행정동명",
    "기준_년분기_코드":"년_분기",
    "서비스_업종_코드_명":"업종명",
    "시간대_00~06_매출_금액": "00~06_매출",
    "시간대_06~11_매출_금액": "06~11_매출",
    "시간대_11~14_매출_금액": "11~14_매출",
    "시간대_14~17_매출_금액": "14~17_매출",
    "시간대_17~21_매출_금액": "17~21_매출",
    "시간대_21~24_매출_금액": "21~24_매출"
})
#이름 재설정 되었나 확인.
dining_sales.columns

Index(['년_분기', '행정동_코드', '행정동명', '서비스_업종_코드', '업종명', '당월매출', '주중_매출_금액',
       '주말_매출_금액', '월요일매출', '화요일매출', '수요일매출', '목요일매출', '금요일매출', '토요일매출',
       '일요일매출', '00~06_매출', '06~11_매출', '11~14_매출', '14~17_매출', '17~21_매출',
       '21~24_매출', '남성매출', '여성매출', '10대_매출금액', '20대_매출금액', '30대_매출금액',
       '40대_매출금액', '50대_매출금액', '60대_매출금액'],
      dtype='object')

In [420]:
#데이터들중 무슨 데이터를 사용할 지 찾기위해 ['업종명']컬럼의 의 벨류들을 찾음.
dining_sales['업종명'].value_counts()

업종명
한식음식점      9306
커피-음료      9070
분식전문점      8935
치킨전문점      8303
호프-간이주점    8205
제과점        8139
패스트푸드점     7875
중식음식점      7833
일식음식점      6314
양식음식점      5445
Name: count, dtype: int64

In [422]:
#서비스 업종 중 내가 보고싶은 5가지를 추려서 "커피-음료","패스트푸드점","중식음식점","일식음식점","양식음식점" 데이터만 찾아서 최종파일인 total_sales에 저장
total = dining_sales.query('업종명 in ["커피-음료","패스트푸드점","중식음식점","일식음식점","양식음식점"]')

# 병합된 데이터 파일 저장.
total.to_csv('Data/Output_Data/서울시_상권분석서비스_total_2019to2024_cp949.csv', index=False, encoding='cp949')
total.head()

,년_분기,행정동_코드,행정동명,서비스_업종_코드,업종명,당월매출,주중_매출_금액,주말_매출_금액,월요일매출,화요일매출,...,17~21_매출,21~24_매출,남성매출,여성매출,10대_매출금액,20대_매출금액,30대_매출금액,40대_매출금액,50대_매출금액,60대_매출금액
22,20191,11110515,청운효자동,CS100004,양식음식점,996290762,720130068,276160694,105992490,139127256,...,372144305,241842885,387041600,416526593,1557262,216362216,275291624,120331565,152169895,37855629
23,20192,11110515,청운효자동,CS100004,양식음식점,1104757459,825979886,278777573,145006121,167094663,...,417439090,217044893,383811169,508179503,2435651,257645723,279089715,170377901,140583887,41857790
24,20193,11110515,청운효자동,CS100004,양식음식점,1183841025,873958509,309882516,161307076,158829203,...,517229518,232399486,399763658,584500779,3696033,300435872,318170061,196080670,121952765,43929038
25,20194,11110515,청운효자동,CS100004,양식음식점,1614881326,1200517873,414363453,154925404,247867109,...,607585824,443137196,581019208,643850312,3026635,398630709,375924792,182377368,184777279,80132736
26,20201,11110515,청운효자동,CS100004,양식음식점,1141669904,830654545,311015359,134460866,181831919,...,481478465,225025742,453346748,487945139,1344292,300047699,277035052,158536136,134282730,70045979


In [404]:
#컬럼 이름들 정리
consump = consump.rename(columns = {
    "식료품_지출_총금액": "식료품지출금액",
    "의류_신발_지출_총금액": "의류신발지출금액",
    "생활용품_지출_총금액": "생활용품지출금액",
    "의료비_지출_총금액": "의료비지출금액",
    "교통_지출_총금액": "교통지출금액",
    "교육_지출_총금액": "교육지출금액",
    "유흥_지출_총금액": "유흥지출금액",
    "여가_문화_지출_총금액": "여가문화지출금액",
    "기타_지출_총금액": "기타지출금액",
    "음식_지출_총금액": "음식지출금액",
    "지출_총금액":"지출총금액",
    "행정동_코드_명":"행정동명",
    
})
#쓸모없는 열 삭제
consump =consump.drop(columns = ["기준_년분기_코드","소득_구간_코드","월_평균_소득_금액"])

In [405]:
#삭제/rename 확인
consump.columns

Index(['행정동_코드', '행정동명', '지출총금액', '식료품지출금액', '의류신발지출금액', '생활용품지출금액', '의료비지출금액',
       '교통지출금액', '교육지출금액', '유흥지출금액', '여가문화지출금액', '기타지출금액', '음식지출금액'],
      dtype='object')

In [423]:
#경로로 잘 저장해놓기
consump.to_csv('Data/Output_Data/New서울시 상권분석서비스.csv', index=False, encoding='cp949')